<a href="https://colab.research.google.com/github/CristhianSeverino/customer-churn-and-contextual-chat/blob/staging/DSMarketWiFi_DataCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Customer Churn and Contextual Chat C4**

> Este **Notebook** es una creación de: **Cristhian Calle Severino**☕.

---

* **GitHub**: https://github.com/CristhianSeverino
* **LinkedIn**: https://www.linkedin.com/in/cristhianandrescalleseverino/

---

Este proyecto **emula un Data Product de Machine Learning (ML) con despliegue en AWS**. Se inspira en la arquitectura **Arana**.

### **Incluye:**

* **Creación de Datos Sintéticos**: Estos datos simulan usuarios de **MallPlaza**, un nombre genérico para un supuesto ficticio, dado que grandes centros comerciales homónimos existen en ciudades de América (como Santiago de Chile y Manizales, donde resido). Esta creación se basa en un supuesto de **Ingeniería y Arquitectura de Datos** que se detalla en el **Road Map**.

---

## **Módulos del Proyecto**📚

El proyecto está estructurado en los siguientes **Módulos**:

> * **Módulo 1 - Creación de Datos Sintéticos**: Emula un proceso **ELT** (Extracción, Carga y Transformación) de ingesta, carga y posterior transformación de datos. El resultado es un *dataset* limpio, listo para el entrenamiento de modelos de ML.
> * **Módulo 2 - EDA (*Exploratory Data Analysis*)**: Análisis de datos exploratorio para la identificación de *insights* previos al ML. Incluye la búsqueda de características de enriquecimiento y el desarrollo de **Feature Engineering**.
> * **Módulo 3 - Entrenamiento y Evaluación del Algoritmo**: Desarrollo del modelo, métricas de ajuste, evaluación de precisión y **Fine Tuning**.
> * **Módulo 4 - Despliegue en AWS**: Implementación y prueba del despliegue en la nube.

---

## **Prerrequisitos y Recomendaciones**🧮

**Nota**: Para este proyecto se emplea **AWS como *cloud platform***.

Es **indispensable** poseer conocimientos en las siguientes tecnologías para comprender a cabalidad los aspectos técnicos:

* **Lenguajes y Bases de Datos**: Python, SQL, VectorDB, Terraform
* **Big Data y ML**: PySpark, Keras, Scikit-learn, PyTorch.
* **Cloud y Herramientas MLOps**: AWS, Git.
* **IA y Desarrollo de Aplicaciones**: LangChain, RAG, Streamlit, Gradio, API.

Para una comprensión general del proyecto, **recomiendo encarecidamente la lectura del Road Map** y una revisión general de la documentación del código. 🦾

> ¡Que disfrutes explorando este proyecto! Exitosa semana. 🐱‍🏍

# **Instalacción E Importación de Librerias // Istall & Import Librarys**📚

In [16]:
!pip install boto3
print("="*150)
print(" "*50+"Boto install OK🐱‍🏍")
print("="*150)

                                                  Boto install OK🐱‍🏍


In [17]:

!pip install awscli
print("="*150)
print(" "*50+"AWS Cli Install OK")
print("="*150)

                                                  AWS Cli Install OK


In [18]:
!pip install faker
print("="*150)
print(" "*50+"Librerias instaladas")
print("="*150)


                                                  Librerias instaladas


In [19]:
# ===================================================   Dependencys or Librarys   ==========================================
# Enviroment Variables
import os
from google.colab import userdata

#Data manipulation and generation
from faker import Faker
import pandas as pd
from datetime import datetime
from datetime import datetime, timedelta
import random
import numpy as np
# AWS
import boto3

print("="*150)
print(" "*50+"Librerias Importadas con Exito ☕")
print("="*150)

                                                  Librerias Importadas con Exito ☕


# **Credenciales // Credentials**🎫

**Ojo Actualizar Credenciales para el rol del Proyecto. Segun Data gobernance**👀

In [20]:
# ==========================================   Variables de entoro o Credentials ==============================================================
# AWS
aws_access_key_id = userdata.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = userdata.get('AWS_SECRET_ACCESS_KEY')
region_name=userdata.get('AWS_REGION')
bucket_name=userdata.get('s3_mallflow')
s3 = boto3.client('s3',
                  aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key,
                  region_name=region_name)
# Git Path
git_path= userdata.get('GIT_PATH')
user_git=userdata.get('USER_GIT')
email_git=userdata.get('EMAIL_GIT')
print("="*150)
print(" "*50+"Credenciales Cargadas")
print("="*150)

                                                  Credenciales Cargadas


# **Generación de Data Set Sintetico // Generate Syntetic Data Set**🤖

In [21]:


def generar_dataset_churn_wifi(n_muestras=1000, output_file='dataset_churn_mall_plaza.csv'):
    fake = Faker('es_CL')
    np.random.seed(42)
    random.seed(42)

    data = {
        'customerId': [fake.uuid4() for _ in range(n_muestras)],
        'name': [fake.name() for _ in range(n_muestras)],
        'gender': np.random.choice(['Male', 'Female', 'Other'], n_muestras, p=[0.485, 0.51, 0.005]),
        'age': np.random.randint(18, 81, n_muestras),
        'visits': np.random.randint(1, 31, n_muestras),
        'avg_session_time': np.random.uniform(5, 120, n_muestras).round(2),
        'complaints': np.random.randint(0, 11, n_muestras),
        'location': [fake.random_element(elements=['Mall Central', 'Mega Tienda', 'Cinemas', 'Electronica', 'Mall Comidas 1', 'Mall Comidas 2', 'Gimnasios', 'Arcades']) for _ in range(n_muestras)],
        'device_type': np.random.choice(['Smartphone', 'Laptop', 'Tablet'], n_muestras, p=[0.85, 0.08, 0.07]),
    }
    df = pd.DataFrame(data)


    df['total_session_time'] = df['visits'] * df['avg_session_time'] + np.random.normal(0, 10, n_muestras)
    df['complaints_per_visit'] = np.where(df['visits'] > 0, df['complaints'] / df['visits'], 0)
    df['log_visits'] = np.log1p(df['visits'])
    bins = [18, 30, 50, 81]; labels = ['Young', 'Adult', 'Senior']
    df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)
    df['high_complaints_flag'] = (df['complaints'] > 5).astype(int)
    max_visits = 30; max_session = 120
    df['engagement_score'] = (df['visits']/max_visits) * (df['avg_session_time']/max_session) * (1 - df['complaints_per_visit'].clip(0,1))
    df['gender_age_interaction'] = df['age'] * (df['gender'] == 'Female').astype(int)
    today = datetime.now()
    df['last_visit_date'] = [today - timedelta(days=np.random.randint(0,31)) for _ in range(n_muestras)]
    df['recency_days'] = (today - df['last_visit_date']).dt.days
    df['tenure_months'] = np.random.randint(6, 61, n_muestras)
    df['data_usage_total'] = df['total_session_time'] * np.random.uniform(0.5, 2.0, n_muestras)

    # Genera churn basado en features
    logit = - (df['visits'] / 30) * 2 - (df['avg_session_time'] / 120) * 2 + (df['complaints'] / 10) * 4 + np.random.normal(0, 0.5, n_muestras)
    p_churn = 1 / (1 + np.exp(-logit))
    df['churn'] = np.where(np.random.rand(n_muestras) < p_churn, 'Yes', 'No')

    # Opcional: One-hot encoding para emular dataset listo para ML
    df = pd.get_dummies(df, columns=['gender', 'location', 'device_type', 'age_group'])

    df.to_csv(output_file, index=False)
    print(f"Dataset Generado 🧮 y guardado en '{output_file}' con {n_muestras} muestras")
    return df

In [22]:
df = generar_dataset_churn_wifi(n_muestras=100000, output_file='churn_abril_mallplaza.csv' )

print(df.head())


Dataset Generado 🧮 y guardado en 'churn_abril_mallplaza.csv' con 100000 muestras
                             customerId                                 name  \
0  93035a22-856f-40dc-80a2-145487d6025c        Josefa Myriam Acevedo Acevedo   
1  048dbbf1-280b-4ee5-a7cc-92c6d2978d55            Doris Cynthia Leal Candia   
2  ba090057-e9a7-4442-8e31-4d569c7c56cc               Valentina Guerra Prado   
3  6d168164-da87-49df-add0-35121f26cb54  Noemí Sabrina Quintanilla Fernández   
4  5444b8d8-5a4d-4619-8219-d4b2a28e7198         Alejandra Josefina Lara Cruz   

   age  visits  avg_session_time  complaints  total_session_time  \
0   51       3             32.57           0           97.120068   
1   25      13             64.52           8          826.446774   
2   79      27             84.52           2         2287.022366   
3   32      28             65.70           1         1828.755038   
4   74      20             27.03           8          543.183295   

   complaints_per_visit  log_

# **Carga de Datos y envio para Downs Stakeholders Bucket de AWS // Load Data for Delivery** 💾

In [23]:
# rear los archivos de configuración de AWS
# Directorio y Archivo de Credenciales
aws_dir = '/root/.aws'
os.makedirs(aws_dir, exist_ok=True)
with open(os.path.join(aws_dir, 'credentials'), 'w') as f:
    f.write(f'[default]\n')
    f.write(f'aws_access_key_id = {aws_access_key_id}\n')
    f.write(f'aws_secret_access_key = {aws_secret_access_key}\n')
# Write 'config'
with open(os.path.join(aws_dir, 'config'), 'w') as f:
    f.write(f'[default]\n')
    f.write(f'region = {region_name}\n')
print("Archivos de cofiguracion de AWs creados Exitosamente ☕")


Archivos de cofiguracion de AWs creados Exitosamente ☕


In [25]:
def upload_to_s3(local_file,s3_path):
    s3.upload_file(local_file, bucket_name, s3_path)
    print(f"Subido {local_file} a s3://{bucket_name}/{s3_path}")
upload_to_s3('/content/customer-churn-and-contextual-chat/churn_abril_mallplaza.csv', 'warehouse-ml/churn_abril_mallplaza/churn_abril_mallplaza.csv')

Subido /content/customer-churn-and-contextual-chat/churn_abril_mallplaza.csv a s3://mall-flow/warehouse-ml/churn_abril_mallplaza/churn_abril_mallplaza.csv


# ***Verifica la Crreacion de los CSV en el icono de carpeta.costado izquiero de colab***

> **Diviertete Explorando este Proyecto**🐱‍👓

🧭Si te fue de ayuda este proyecto, pasa por mi linkedin y cuentame como te ayudo ☕
